In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [2]:
torch.cuda.is_available()
torch.backends.cudnn.enabled

NameError: name 'torch' is not defined

In [ ]:
PATH = 'data/dogbreed/'
sz = 200
arch=resnext50 #or 34
bs = 58

In [ ]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n) #takes default (0.2?) amount of indices from 0 to n from training set for val

In [3]:
label_df = pd.read_csv(label_csv)
label_df.head()

NameError: name 'pd' is not defined

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name = 'test', 
                                    val_idxs = val_idxs, suffix = '.jpg', tfms = tfms, bs = bs)
fn = PATH+data.trn_ds.fnames[0]; fn

In [ ]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name = 'test', num_workers = 4,
                                        val_idxs = val_idxs, suffix = '.jpg', tfms = tfms, bs = bs)
    #return data if sz > 300 else data.resize(340, 'tmp')
    return data

In [ ]:
data = get_data(sz, bs)
learn = ConvLearner.pretrained(arch, data, precompute = True)
learn.fit(1e-2, 5)

In [ ]:
learn.precompute = False
learn.fit(1e-2, 5, cycle_len = 1)

In [ ]:
learn.set_data(get_data(299, bs))
learn.fit(1e-2, 6, cycle_len = 1, cycle_mult = 2)

In [4]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds, y), metrics.log_loss(y, probs)

NameError: name 'learn' is not defined

In [ ]:
submit = pd.DataFrame(probs)
submit.columns = data.classes
submit.insert(0, 'id', [Id[5:-4] for Id in data.test_ds.fnames]) #skip prefix and suffix 5:-4

In [ ]:
SUBM = f'{PATH}subm/'
os.makedirs(SUBM, exist_ok = True)
ds.to_csv(f'{SUBM}submit.csv', index = False)
FileLink(f'{SUBM}submit.csv')